In [17]:
# import torch
# from bilstm_crf import BiLSTMCRF
from torch import optim
from utils import *
from tqdm import tqdm
from sklearn.preprocessing import MultiLabelBinarizer


# import evaluation metrics 
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.model_selection import train_test_split

torch.manual_seed(42)

In [18]:
import torch
import torch.nn as nn
from TorchCRF import CRF

class BiLSTMCRF(nn.Module):
    def __init__(self, vocab_size, word_embedding_dim, hidden_dim, output_dim):
        
        super(BiLSTMCRF, self).__init__()
        # hyperparameters
        self.word_embedding_dim = word_embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.output_dim = output_dim

        # model layers
        self.word_embedding = nn.Embedding(vocab_size, word_embedding_dim)
        self.lstm = nn.LSTM(word_embedding_dim, hidden_dim // 2, bidirectional=True, dropout=0.2)
        self.hidden_to_tag = nn.Linear(hidden_dim, output_dim)
        self.crf = CRF(output_dim)

    def __init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2), torch.randn(2, 1, self.hidden_dim // 2)) # initialize hidden state

    def __get_lstm_features(self, sentence):
        self.hidden = self.__init_hidden()
        word_embeddings = self.word_embedding(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(word_embeddings, self.hidden)
        lstm_features = self.hidden_to_tag(lstm_out)
        return lstm_features

    def neg_log_likelihood(self, sentence, tags):
        features = self.__get_lstm_features(sentence) # emissions from the lstm layer
        tags = tags.view(-1, 1)
        loss = -self.crf(features, tags)
        return loss

    def forward(self, sentence):
        lstm_features = self.__get_lstm_features(sentence) # emissions from the lstm layer
        tag_sequence = self.crf.decode(lstm_features)
        return tag_sequence
    

In [19]:
def read_dataset():
    data = pd.read_csv('../ner_dataset/ner_dataset.csv', encoding='latin1')

    # remove white spaces from column names
    data.columns = data.columns.str.strip()

    print(data.columns)
    # Group by 'Sentence #' and aggregate
    grouped_data = data.groupby('Sentence #').agg({
        'Word': lambda x: ''.join(x),  # Join words into a single sentence
        'Tag': lambda x: list(x.str.strip()),       # Collect tags into a list
        'Intent': lambda x: list(x.str.strip().str.replace('_', ' '))     # Collect intents into a list
    }).reset_index()  # Reset index to make 'Sentence #' a regular column

    return data, grouped_data


def prepare_data(dataframe):
    dataset = []
    for _, row in dataframe.iterrows():
        sentence = row['Word'][1:]
        tags = row['Tag']
        intents = row['Intent']
        dataset.append((sentence, tags, intents[0]))

    return dataset

In [20]:
# Format data
data, goruped_data = read_dataset()
print(data["Intent"].unique())

goruped_data.head()

Index(['Sentence #', 'Word', 'Tag', 'Intent'], dtype='object')
[' array operation' ' assertion' ' assignment operation'
 ' bitwise operation' ' casting' ' class declaration' ' comment'
 ' conditional operation' ' constant declaration' ' file system'
 ' for loop' ' function declaration' ' git operation' ' ide operation'
 ' input' ' interactive commands' ' libraries' ' mathematical operation'
 ' membership operation' ' output' ' variable declaration' ' while loop']


,Sentence #,Word,Tag,Intent
0,0,append the value 12345.6789 to the list packe...,"[B-OPERATION, O, O, B-ELEMENT, O, O, B-ARRAY, ...","[array operation, array operation, array opera..."
1,1,push the value false to the list inventories,"[B-OPERATION, O, O, B-ELEMENT, O, O, O, B-ARRAY]","[array operation, array operation, array opera..."
2,2,append the value -456 to the array player scores,"[B-OPERATION, O, O, B-ELEMENT, O, O, O, B-ARRA...","[array operation, array operation, array opera..."
3,3,push the value 3.14159 to the list links,"[B-OPERATION, O, O, B-ELEMENT, O, O, O, B-ARRAY]","[array operation, array operation, array opera..."
4,4,include the value Back End Development to the...,"[B-OPERATION, O, O, B-ELEMENT, I-ELEMENT, I-EL...","[array operation, array operation, array opera..."


In [21]:
training_data = prepare_data(goruped_data)

sentences = [sentence for sentence, _, _ in training_data]
tags = [tag for _, tag, _ in training_data]
intents = set([intent for _, _, intent in training_data])

intents

{'array operation',
 'assertion',
 'assignment operation',
 'bitwise operation',
 'casting',
 'class declaration',
 'comment',
 'conditional operation',
 'constant declaration',
 'file system',
 'for loop',
 'function declaration',
 'git operation',
 'ide operation',
 'input',
 'interactive commands',
 'libraries',
 'mathematical operation',
 'membership operation',
 'output',
 'variable declaration',
 'while loop'}

In [22]:
separate_data = {} # a dictionary to store the data for each intent

# separate training data by intent
for intent in intents:
    separate_data[intent] = [(sentence, tag) for sentence, tag, _intent in training_data if _intent == intent]

for intent in separate_data:
    print(f'Intent: {intent}')
    print(f'Number of sentences: {len(separate_data[intent])}')
    print(f'Example: {separate_data[intent][0]}')
    print('\n')

Intent: assignment operation
Number of sentences: 80
Example: ('end date equals clustering algorithms', ['B-LHS', 'I-LHS', 'O', 'B-RHS', 'I-RHS'])


Intent: git operation
Number of sentences: 80
Example: ('stage', ['B-ACTION'])


Intent: interactive commands
Number of sentences: 220
Example: ('go get all the files in the project', ['O', 'O', 'O', 'O', 'B-TYPE', 'O', 'O', 'O'])


Intent: bitwise operation
Number of sentences: 97
Example: ('perform a bitwise xor on creation date and 150', ['O', 'O', 'O', 'B-OPERATOR', 'O', 'B-OPERAND', 'I-OPERAND', 'O', 'B-OPERAND'])


Intent: for loop
Number of sentences: 120
Example: ('create a for loop from -30 to -10 with variable item', ['O', 'O', 'O', 'O', 'O', 'B-START', 'O', 'B-END', 'O', 'O', 'B-VAR'])


Intent: ide operation
Number of sentences: 290
Example: ('launch terminal', ['B-ACTION', 'B-TYPE'])


Intent: comment
Number of sentences: 50
Example: ('add a this is a log comment', ['O', 'I-COMMENT', 'I-COMMENT', 'I-COMMENT', 'O', 'I-COMMENT',

In [23]:
separate_word_to_index = {}
separate_tag_to_index = {}
separate_index_to_tag = {}

for intent in separate_data:
    words = [word for sentence, _ in separate_data[intent] for word in sentence.split()]
    tags = [tag for _, tag in separate_data[intent] for tag in tag]

    word_to_index = {word: i + 1 for i, word in enumerate(set(words))}
    tag_to_index = {tag: i + 1 for i, tag in enumerate(set(tags))}

    word_to_index['<UNK>'] = 0
    tag_to_index['<UNK>'] = 0

    separate_word_to_index[intent] = word_to_index
    separate_tag_to_index[intent] = tag_to_index

separate_index_to_tag = {intent: {i: tag for tag, i in separate_tag_to_index[intent].items()} for intent in separate_tag_to_index}

for intent in separate_data:
    print(f'Intent: {intent}')
    print(f'Number of unique words: {len(separate_word_to_index[intent])}')
    print(f'Number of unique tags: {len(separate_tag_to_index[intent])}')
    print('\n')

Intent: assignment operation
Number of unique words: 178
Number of unique tags: 6


Intent: git operation
Number of unique words: 36
Number of unique tags: 5


Intent: interactive commands
Number of unique words: 22
Number of unique tags: 3


Intent: bitwise operation
Number of unique words: 144
Number of unique tags: 6


Intent: for loop
Number of unique words: 141
Number of unique tags: 9


Intent: ide operation
Number of unique words: 134
Number of unique tags: 7


Intent: comment
Number of unique words: 59
Number of unique tags: 4


Intent: constant declaration
Number of unique words: 305
Number of unique tags: 7


Intent: conditional operation
Number of unique words: 283
Number of unique tags: 9


Intent: assertion
Number of unique words: 117
Number of unique tags: 8


Intent: variable declaration
Number of unique words: 299
Number of unique tags: 7


Intent: function declaration
Number of unique words: 173
Number of unique tags: 7


Intent: class declaration
Number of unique word

In [24]:
with open('../ner_dataset/intent_to_tags.json', 'r') as f:
    intent_to_tags = json.load(f)

final_intent_to_tags = {}

for intent in intent_to_tags.keys():
    final_intent_to_tags[intent] = []
    for tag in intent_to_tags[intent]:
        if tag == 'O':
            final_intent_to_tags[intent].append(tag)
        else:
            final_intent_to_tags[intent].append('B-' + tag)
            final_intent_to_tags[intent].append('I-' + tag)

        
final_intent_to_tags

{'array operation': ['B-ARRAY',
  'I-ARRAY',
  'B-OPERATION',
  'I-OPERATION',
  'B-ELEMENT',
  'I-ELEMENT',
  'O'],
 'assertion': ['B-VAR',
  'I-VAR',
  'B-VAL',
  'I-VAL',
  'B-CONDITION',
  'I-CONDITION',
  'O'],
 'assignment operation': ['B-LHS', 'I-LHS', 'B-RHS', 'I-RHS', 'O'],
 'bitwise operation': ['B-OPERATOR',
  'I-OPERATOR',
  'B-OPERAND',
  'I-OPERAND',
  'O'],
 'casting': ['B-VAR', 'I-VAR', 'B-TYPE', 'I-TYPE', 'O'],
 'class declaration': ['B-CLASS', 'I-CLASS', 'O'],
 'comment': ['B-COMMENT', 'I-COMMENT', 'O'],
 'conditional operation': ['B-LHS',
  'I-LHS',
  'B-RHS',
  'I-RHS',
  'B-CONDITION',
  'I-CONDITION',
  'B-LOG',
  'I-LOG',
  'O'],
 'constant declaration': ['B-VAR',
  'I-VAR',
  'B-VAL',
  'I-VAL',
  'B-TYPE',
  'I-TYPE',
  'O'],
 'file system': ['B-FILE',
  'I-FILE',
  'B-ACTION',
  'I-ACTION',
  'B-DIR',
  'I-DIR',
  'O'],
 'for loop': ['B-START',
  'I-START',
  'B-END',
  'I-END',
  'B-STEP',
  'I-STEP',
  'B-VAR',
  'I-VAR',
  'B-COLLECTION',
  'I-COLLECTION',


In [67]:
# training data for each intent
# training_data = {intent: [x_train, y_train, x_test, y_test]}
training_data = {} 
for intent in separate_data:
    sentences = [sentence for sentence, _ in separate_data[intent]]
    tags = [tag for _, tag in separate_data[intent]]
    x_train, x_test, y_train, y_test = train_test_split(sentences, tags, test_size=0.1, random_state=42)
    training_data[intent] = [x_train, y_train, x_test, y_test]


72 72 8 8


In [26]:
models = {}

for intent in final_intent_to_tags.keys():
    models[intent] = []
    models[intent].append(BiLSTMCRF(vocab_size=len(separate_word_to_index[intent]), word_embedding_dim=50, hidden_dim=64, output_dim=len(separate_tag_to_index[intent])))
    models[intent].append(optim.SGD(models[intent][0].parameters(), lr=0.01, weight_decay=1e-4))

c:\Users\yazmi\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [27]:
# Check predictions before training
# tarining data {
#     intent: [x_train, y_train, x_test, y_test]
# }
with torch.no_grad():
    for intent in intents:
        print(f'Intent: {intent}')
        precheck_sent = prepare_sequence(training_data[intent][0][0].split(), separate_word_to_index[intent])
        precheck_tag = prepare_sequence(training_data[intent][1][0], separate_tag_to_index[intent])
        print("Actual tags:", precheck_tag)
        print("Predicted tags:", models[intent][0](precheck_sent))


Intent: assignment operation
Actual tags: tensor([2, 4, 3, 1])
Predicted tags: [[0, 4, 4, 2]]
Intent: git operation
Actual tags: tensor([4])
Predicted tags: [[2]]
Intent: interactive commands
Actual tags: tensor([1, 1, 1, 2])
Predicted tags: [[2, 0, 0, 0]]
Intent: bitwise operation
Actual tags: tensor([3, 3, 3, 2, 5, 3, 3, 4, 1, 3, 4])
Predicted tags: [[2, 5, 0, 4, 4, 1, 2, 5, 1, 1, 1]]
Intent: for loop
Actual tags: tensor([3, 3, 3, 3, 3, 3, 2, 3, 7, 3, 3, 6, 1])
Predicted tags: [[7, 8, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7]]
Intent: ide operation
Actual tags: tensor([6, 2, 4])
Predicted tags: [[6, 0, 6]]
Intent: comment
Actual tags: tensor([3, 1, 1, 2])
Predicted tags: [[3, 1, 0, 3]]
Intent: constant declaration
Actual tags: tensor([3, 3, 3, 3, 3, 4, 3, 3, 5])
Predicted tags: [[4, 2, 6, 4, 2, 4, 5, 4, 2]]
Intent: conditional operation
Actual tags: tensor([4, 3, 6, 4, 1, 8, 4, 5, 7, 2, 4, 4, 4, 1, 4, 4, 4])
Predicted tags: [[6, 4, 4, 2, 3, 6, 6, 8, 7, 4, 4, 4, 2, 3, 5, 8, 7]]
Intent: asserti

In [28]:
# training loop
def train(model, optimizer, x_train, y_train,epochs=10):
    for epoch in tqdm(range(epochs)):
        total_loss = 0
        for sentence, tags in zip(x_train, y_train):
            # Step 1. Remember that Pytorch accumulates gradients.
            # We need to clear them out before each instance
            model.zero_grad()
        
            sentence = prepare_sequence(sentence.split(), separate_word_to_index[intent])
            target_tags = prepare_sequence(tags, separate_tag_to_index[intent])

            # Step 3. Run our forward pass.
            loss = model.neg_log_likelihood(sentence, target_tags)

            total_loss += loss.item()

            # Step 4. Compute the loss, gradients, and update the parameters by
            loss.backward()
            optimizer.step()
        
        print(f"Epoch: {epoch}, Loss: {total_loss / len(x_train)}")

In [ ]:
for intent in intents:
    print(f'Training model for intent: {intent}')
    x_train, y_train = training_data[intent][0], training_data[intent][1]
    train(models[intent][0], models[intent][1], x_train, y_train, epochs=75)

In [30]:
# Check predictions before training
# tarining data {
#     intent: [x_train, y_train, x_test, y_test]
# }
with torch.no_grad():
    for intent in intents:
        print(f'Intent: {intent}')
        precheck_sent = prepare_sequence(training_data[intent][0][0].split(), separate_word_to_index[intent])
        precheck_tag = prepare_sequence(training_data[intent][1][0], separate_tag_to_index[intent])
        print("Actual tags:", precheck_tag)
        print("Predicted tags:", models[intent][0](precheck_sent))


Intent: assignment operation
Actual tags: tensor([2, 4, 3, 1])
Predicted tags: [[2, 4, 3, 1]]
Intent: git operation
Actual tags: tensor([4])
Predicted tags: [[4]]
Intent: interactive commands
Actual tags: tensor([1, 1, 1, 2])
Predicted tags: [[1, 1, 1, 2]]
Intent: bitwise operation
Actual tags: tensor([3, 3, 3, 2, 5, 3, 3, 4, 1, 3, 4])
Predicted tags: [[3, 3, 3, 2, 5, 3, 3, 4, 1, 3, 4]]
Intent: for loop
Actual tags: tensor([3, 3, 3, 3, 3, 3, 2, 3, 7, 3, 3, 6, 1])
Predicted tags: [[3, 3, 3, 3, 3, 3, 2, 3, 7, 3, 3, 6, 1]]
Intent: ide operation
Actual tags: tensor([6, 2, 4])
Predicted tags: [[6, 2, 4]]
Intent: comment
Actual tags: tensor([3, 1, 1, 2])
Predicted tags: [[3, 1, 1, 2]]
Intent: constant declaration
Actual tags: tensor([3, 3, 3, 3, 3, 4, 3, 3, 5])
Predicted tags: [[3, 3, 3, 3, 3, 4, 3, 3, 5]]
Intent: conditional operation
Actual tags: tensor([4, 3, 6, 4, 1, 8, 4, 5, 7, 2, 4, 4, 4, 1, 4, 4, 4])
Predicted tags: [[4, 3, 6, 4, 1, 8, 4, 5, 7, 2, 4, 4, 4, 1, 4, 4, 4]]
Intent: asserti

In [31]:
def predict(model, test_data):
    model.eval()
    predictions = [] # a dictionary to store the predictions for each intent
    with torch.no_grad():
        for sentence in test_data:
            precheck_sent = prepare_sequence(sentence.split(), separate_word_to_index[intent])
            predictions.append(model(precheck_sent)[0])
    return predictions

In [51]:
# evaluation 
def evaluate(y_true, y_pred):
    mlb = MultiLabelBinarizer()
    y_true_binarized = mlb.fit_transform(y_true)
    y_pred_binarized = mlb.transform(y_pred)
    
    f1 = f1_score(y_true_binarized, y_pred_binarized, average='weighted')
    precision = precision_score(y_true_binarized, y_pred_binarized, average='weighted')
    recall = recall_score(y_true_binarized, y_pred_binarized, average='weighted')
    accuracy = accuracy_score(y_true_binarized, y_pred_binarized)
    
    return f1, precision, recall, accuracy

In [33]:
for intent in intents:
    print(f'Intent: {intent}')
    x_test, y_test = training_data[intent][2], training_data[intent][3]
    predictions = predict(models[intent][0], x_test)

    mapped_predictions = [[separate_index_to_tag[intent][tag] for tag in tags] for tags in predictions]

    f1, precision, recall, accuracy = evaluate(y_test, mapped_predictions)
    print("F1 Score:", f1)
    print("Precision:", precision)
    print("Recall:", recall)
    print("Accuracy:", accuracy)
    print("----------------------------------------------------------------------")

Intent: assignment operation
F1 Score: 0.9529411764705882
Precision: 0.9742647058823529
Recall: 0.9411764705882353
Accuracy: 0.875
----------------------------------------------------------------------
Intent: git operation
F1 Score: 0.9751552795031057
Precision: 1.0
Recall: 0.9565217391304348
Accuracy: 0.875
----------------------------------------------------------------------
Intent: interactive commands
F1 Score: 1.0
Precision: 1.0
Recall: 1.0
Accuracy: 1.0
----------------------------------------------------------------------
Intent: bitwise operation
F1 Score: 1.0
Precision: 1.0
Recall: 1.0
Accuracy: 1.0
----------------------------------------------------------------------
Intent: for loop
F1 Score: 0.9918330308529946
Precision: 0.9844827586206897
Recall: 1.0
Accuracy: 0.9166666666666666
----------------------------------------------------------------------
Intent: ide operation
F1 Score: 0.9674215587044533
Precision: 0.9636222910216719
Recall: 0.9736842105263158
Accuracy: 0.896

c:\Users\yazmi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\yazmi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:895: UserWarning: unknown class(es) ['B-VAR', 'I-VAR'] will be ignored
  warnings.warn(
c:\Users\yazmi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\yazmi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Preci

F1 Score: 0.9563761620800771
Precision: 0.9815873015873016
Recall: 0.9428571428571428
Accuracy: 0.625
----------------------------------------------------------------------
Intent: assertion
F1 Score: 0.7246376811594203
Precision: 0.8695652173913043
Recall: 0.6956521739130435
Accuracy: 0.0
----------------------------------------------------------------------
Intent: variable declaration
F1 Score: 0.9820873205741627
Precision: 1.0
Recall: 0.9659090909090909
Accuracy: 0.9166666666666666
----------------------------------------------------------------------
Intent: function declaration
F1 Score: 0.9795150501672242
Precision: 0.9821428571428571
Recall: 0.9791666666666666
Accuracy: 0.9166666666666666
----------------------------------------------------------------------
Intent: class declaration
F1 Score: 0.8253968253968255
Precision: 0.8888888888888888
Recall: 0.7777777777777778
Accuracy: 0.5
----------------------------------------------------------------------
Intent: libraries
F1 Score

In [74]:
intent = 'file system'
inp = 'create new file test'
label = ['B-ACTION' ,'O', 'O', 'B-FILE']
constant_model = models[intent][0]
constant_model.eval()
predictions = [] # a dictionary to store the predictions for each intent
with torch.no_grad():
    precheck_sent = prepare_sequence(inp.split(), separate_word_to_index[intent])
    pred = constant_model(precheck_sent)[0]
    print(pred)

mapped_predictions = [separate_index_to_tag[intent][tag] for tag in pred]

print("Actual tags:", label)
print("Predicted tags:", mapped_predictions)

f1, precision, recall, accuracy = evaluate(label, mapped_predictions)
print("F1 Score:", f1)
print("Precision:", precision)
print("Recall:", recall)
print("Accuracy:", accuracy)
print("----------------------------------------------------------------------")

[4, 2, 2, 2]
Actual tags: ['B-ACTION', 'O', 'O', 'B-FILE']
Predicted tags: ['B-ACTION', 'O', 'O', 'O']
F1 Score: 0.6607142857142856
Precision: 0.765625
Recall: 0.625
Accuracy: 0.75
----------------------------------------------------------------------


c:\Users\yazmi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [35]:
# Check predictions before training
test_example_1 = ("create a variable called ay khara and set it to yasmine ghanem", ["O", "O", "O", "O", "B-VAR", "I-VAR", "O", "O", "O", "B-VAL", "I-VAL"], "variable declaration")
test_example_2 = ("cast x to an int", ["O", "B-VAR", "O", "O", "B-CAST_TYPE"], "casting")
test_example_3 = ("import the pandas library", ["O", "O", "B-LIB_NAME", "O"], "libraries")
test_example_4 = ('write a new comment multiply the emissions and transitions', ['O', 'O', 'O', 'B-COMMENT', 'I-COMMENT', 'I-COMMENT', 'I-COMMENT', 'I-COMMENT' ], 'comment')
test_example_5 = ('define a new class person', ['O', 'O', 'O', 'O', 'B-CLASS_NAME'], 'class declaration')
test_example_6 = ('create a new function named add that takes parameters x and y', ['O', 'O', 'O', 'O', 'O', 'B-FUNC', 'O', 'O', 'O', 'B-PARAM', 'O', 'B-PARAM'], 'function declaration')
test_example_7 = ('create a new function called add with parameters num1 and num2', ['O', 'O', 'O', 'O', 'O', 'B-FUNC', 'O', 'O', 'O', 'B-PARAM', 'O', 'B-PARAM'], 'function declaration')

test_data = [test_example_1, test_example_2, test_example_3, test_example_4, test_example_5, test_example_6, test_example_7]

for index, example in enumerate(test_data):
    print("Sentence:", example[0])
    print("Intent:", example[2])

    predictions = predict(models[example[2]][0], [example[0]])

    mapped_predictions = [[separate_index_to_tag[example[2]][tag] for tag in tags] for tags in predictions]

    print("Actual tags:", example[1])
    print("Predicted tags:", mapped_predictions[0])

    evaluated = evaluate([example[1]], mapped_predictions)

    print("----------------------------------------------------------------------------------------------------")

# predictions = predict(test_model, test_data)
# labels = []
# predicted_tags = []

# for index, prediction in enumerate(predictions):
#     print("Test Example:", index)
#     mapped_tags = [all_tags[tag] for tag in prediction]
#     predicted_tags.append(mapped_tags)
#     labels.append(test_data[index][1])
#     print("Actual tags:", test_data[index][1])
#     print("Predicted tags:", mapped_tags)

Sentence: create a variable called ay khara and set it to yasmine ghanem
Intent: variable declaration
Actual tags: ['O', 'O', 'O', 'O', 'B-VAR', 'I-VAR', 'O', 'O', 'O', 'B-VAL', 'I-VAL']
Predicted tags: ['B-VAL', 'I-VAL', 'B-VAR', 'I-VAR', 'B-TYPE', 'O', 'B-VAR', 'I-VAR', 'B-VAR', 'I-VAR', 'O', 'B-TYPE']
----------------------------------------------------------------------------------------------------
Sentence: cast x to an int
Intent: casting
Actual tags: ['O', 'B-VAR', 'O', 'O', 'B-CAST_TYPE']
Predicted tags: ['O', 'O', 'B-VAR', 'I-VAR', 'O']
----------------------------------------------------------------------------------------------------
Sentence: import the pandas library
Intent: libraries


c:\Users\yazmi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:895: UserWarning: unknown class(es) ['B-TYPE'] will be ignored
  warnings.warn(
c:\Users\yazmi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:895: UserWarning: unknown class(es) ['I-VAR'] will be ignored
  warnings.warn(
c:\Users\yazmi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


IndexError: index out of range in self

In [38]:
# save models
for intent in intents:
    torch.save(models[intent][0].state_dict(), f"../models/separate_ner_models/{intent}_model.pth")

Try separate data with the contsrained 